In [ ]:
import sys
sys.path.append('/mnt/cocoapi/PythonAPI/')

from pycocotools.coco import COCO
import os, sys, zipfile
import urllib.request
import shutil
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pylab
import json
from pathlib import Path
import csv
import cv2

In [ ]:
source_dir = '/mnt/data/warm_up_train_20180222'
output_dir = '/mnt/data/fai_kp_coco'
image_set = 'train'
data_anno_path = Path(source_dir, 'Annotations', image_set + '.csv')
output_anno_path = Path(output_dir, 'annotations', image_set + '.json')
assert data_anno_path.exists(), "data_anno_path not exists"

In [ ]:
train_anno_op = open(data_anno_path.as_posix(), 'r')
csv_reader = csv.DictReader(train_anno_op)
data_raw = [row for row in csv_reader]
anno_dict = data_raw[0]
print(anno_dict['image_id'])

In [ ]:
data_coco={}

categories=[]
category = {}
category['supercategory'] = anno_dict['image_category']
category['id'] = 1
category['name']= anno_dict['image_category']
categories.append(category)


data_coco['categories'] = categories

In [ ]:
images=[]

image={}
image_path = anno_dict['image_id']
ab_image_path = Path(source_dir, image_path)
assert ab_image_path.exists(), 'Path does not exist: {}'.format(ab_image_path)
image_raw = cv2.imread(ab_image_path.as_posix())
image['height'], image['width'], _ = image_raw.shape
image_name = image_path.split('/')[-1]
image['file_name']=image_name
image['id']=0 

images.append(image)

data_coco['images']=images

In [ ]:
annotations=[]
annotation={}

keypoints_name = ['neckline_left', 'neckline_right', 'center_front', 'shoulder_left', 'shoulder_right', 'armpit_left', 'armpit_right', 'waistline_left', 'waistline_right', 'cuff_left_in', 'cuff_left_out', 'cuff_right_in', 'cuff_right_out', 'top_hem_left', 'top_hem_right', 'waistband_left', 'waistband_right', 'hemline_left', 'hemline_right', 'crotch', 'bottom_left_in', 'bottom_left_out', 'bottom_right_in', 'bottom_right_out']
anno_kp = np.array([anno_dict.get(key).split('_') for key in keypoints_name]).astype(np.int16)
anno_kp = anno_kp[np.where(anno_kp[:, 2]>=0)].astype(np.uint16)
xmax, ymax, _ = anno_kp.max(axis=0)
xmin, ymin, _ = anno_kp.min(axis=0)
bbox = np.array([xmin, ymin, xmax-xmin, ymax-ymin]).tolist()

annotation['segmentation']=[]
annotation['iscrowd']=0
annotation['image_id']=image['id']
# annotation['bbox']=[str(xmin), str(ymin), str(xmax-xmin), str(ymax-ymin)]
annotation['bbox']=bbox
annotation['category_id']=1
annotation['id']=1

annotations.append(annotation)
data_coco['annotations']=annotations

In [ ]:
pose_name_seq = ["neckline_left", "neckline_right", "center_front", "shoulder_left",
                  "shoulder_right", "armpit_left", "armpit_right", "waistline_left",
                  "waistline_right", "cuff_left_in", "cuff_left_out", "cuff_right_in",
                  "cuff_right_out", "top_hem_left", "top_hem_right", "waistband_left",
                  "waistband_right", "hemline_left", "hemline_right", "crotch",
                  "bottom_left_in", "bottom_left_out", "bottom_right_in", "bottom_right_out"]
pose_id_dict = {
  "neckline_left":1, "neckline_right":2, "center_front":3, "shoulder_left":4,
  "shoulder_right":5, "armpit_left":6, "armpit_right":7, "waistline_left":8,
  "waistline_right":9, "cuff_left_in":10, "cuff_left_out":11, "cuff_right_in":12,
  "cuff_right_out":13, "top_hem_left":14, "top_hem_right":15, "waistband_left":16,
  "waistband_right":17, "hemline_left":18, "hemline_right":19, "crotch":20,
  "bottom_left_in":21, "bottom_left_out":22, "bottom_right_in":23, "bottom_right_out":24
}
keypoints_name = ['neckline_left', 'neckline_right', 'center_front', 'shoulder_left', 'shoulder_right', 'armpit_left', 'armpit_right', 'waistline_left', 'waistline_right', 'cuff_left_in', 'cuff_left_out', 'cuff_right_in', 'cuff_right_out', 'top_hem_left', 'top_hem_right', 'waistband_left', 'waistband_right', 'hemline_left', 'hemline_right', 'crotch', 'bottom_left_in', 'bottom_left_out', 'bottom_right_in', 'bottom_right_out']
sorted_blouse_keys = ['neckline_left', 'shoulder_left', 'cuff_left_out', 'cuff_left_in', 'armpit_left', 'top_hem_left', 'top_hem_right', 'armpit_right', 'cuff_right_in', 'cuff_right_out', 'shoulder_right', 'neckline_right', 'center_front']
anno_kp = np.array([anno_dict.get(key).split('_') for key in sorted_blouse_keys]).astype(np.int16)
visable_mask_anno_kp = anno_kp[np.where(anno_kp[:, 2]>=0)].astype(np.float)

print(len(np.where(anno_kp[:, 2] >= 0)[0]))
print(len(np.where(anno_kp[:, 2] > 0)[0]))
print(anno_kp[:, :2])


[ values[0] for values in visable_mask_anno_kp]
mask_op = visable_mask_anno_kp[:, :2].flatten()
print(data_coco['annotations'][0])
data_coco['annotations'][0]['segmentation'] = [mask_op.tolist()]

In [ ]:
json.dump(data_coco, output_anno_path.open(mode='w+'), indent=4)

In [ ]:
coco=COCO(output_anno_path.as_posix())

In [ ]:
imgIds = coco.getImgIds()

In [ ]:
coco.loadImgs(0)

In [ ]:
img_anno = coco.loadImgs(0)[0]
img_path = Path(output_dir, 'images', image_set, img_anno['file_name'])
print(img_path)
assert img_path.exists(), 'aaa'

In [ ]:
img = cv2.imread(img_path.as_posix())
plt.axis('off')
plt.imshow(img)
plt.show()

In [ ]:
visable_mask_anno_kp[:, :2]
kp = visable_mask_anno_kp[:, :2][0]
[cv2.circle(img, (int(kp[0]), int(kp[1])), radius=5, color=(255, 0, 0), thickness=3) for kp in visable_mask_anno_kp[:, :2]]
plt.axis('off')
plt.imshow(img)
plt.show()

In [ ]:
visable_mask_anno_kp[:, :2]
kp = visable_mask_anno_kp[:, :2][0]
[cv2.circle(img, (int(kp[0]), int(kp[1])), radius=5, color=(255, 0, 0), thickness=3) for kp in visable_mask_anno_kp[:, :2]]
plt.axis('off')
plt.imshow(img)
plt.show()

In [ ]:
annIds = coco.getAnnIds(imgIds=[0])
anns = coco.loadAnns(annIds)
anns[0]['bbox']
xmin, ymin, width, height = [int(i) for i in anns[0]['bbox']]
xmin, ymin, width, height = anns[0]['bbox']

In [ ]:
cv2.rectangle(img, (xmin, ymin), (xmin+width, ymin+height), (255, 0, 0), 2)
plt.axis('off')
plt.imshow(img)
plt.show()

In [ ]:
# load and display instance annotations
# 加载实例掩膜
# catIds = coco.getCatIds(catNms=['person','dog','skateboard']);
# catIds=coco.getCatIds()
catIds=[]
for ann in coco.dataset['annotations']:
    if ann['image_id']==0:
        catIds.append(ann['category_id'])
print(catIds)

In [ ]:
plt.imshow(img); plt.axis('off')
annIds = coco.getAnnIds(imgIds=0, catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

In [ ]:
data_coco['categories'][0]['keypoints'] = ["neckline_left", "neckline_right", "center_front", "shoulder_left",
                        "shoulder_right", "armpit_left", "armpit_right", "waistline_left",
                        "waistline_right", "cuff_left_in", "cuff_left_out", "cuff_right_in",
                        "cuff_right_out", "top_hem_left", "top_hem_right", "waistband_left",
                        "waistband_right", "hemline_left", "hemline_right", "crotch",
                        "bottom_left_in", "bottom_left_out", "bottom_right_in", "bottom_right_out"]
data_coco['categories'][0]['skeleton'] = [[1,2], [1,3], [2,3], [1,4], [4,11], [6,7], [6,10], [2,5], [5,13],
                   [7,12], [4,6], [5,7], [6,14], [14,15], [7,15], [6,8], [7,9], [16,17], [8,18],
                   [9,19], [16,18], [17, 19], [16, 22], [17, 24], [16, 20], [17, 20],
                   [20, 21], [20, 23]]
print(data_coco['categories'])

In [ ]:
anno_keys = ["neckline_left", "neckline_right", "center_front", "shoulder_left",
                        "shoulder_right", "armpit_left", "armpit_right", "waistline_left",
                        "waistline_right", "cuff_left_in", "cuff_left_out", "cuff_right_in",
                        "cuff_right_out", "top_hem_left", "top_hem_right", "waistband_left",
                        "waistband_right", "hemline_left", "hemline_right", "crotch",
                        "bottom_left_in", "bottom_left_out", "bottom_right_in", "bottom_right_out"]
anno_kp = np.array([anno_dict.get(key).split('_') for key in anno_keys]).astype(np.int16).tolist()
anno_kps = [[aa[0], aa[1], aa[2]+1] for aa in anno_kp]
data_coco['annotations'][0]['keypoints'] = [ p for kp in anno_kps for p in kp]
data_coco['annotations'][0]['segmentation'] = []
print(data_coco['annotations'])

In [ ]:
json.dump(data_coco, output_anno_path.open(mode='w'), indent=4)

In [ ]:
coco=COCO(output_anno_path.as_posix())
img = cv2.imread(img_path.as_posix())
plt.imshow(img); plt.axis('off')
annIds = coco.getAnnIds(imgIds=0, catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

In [ ]:
plt.axis('off')
plt.imshow(img)
plt.show()